In [133]:
import requests
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

In [183]:
df = pd.read_csv('stock_headlines.csv')
prices = pd.read_csv('stock_prices.csv')
volumes = pd.read_csv('stock_volumes.csv')
filings = pd.read_csv('filing_dates.csv')

In [186]:
df['date'] = pd.to_datetime(pd.to_datetime(df.date).dt.date)
prices['date'] = pd.to_datetime(prices.date)
volumes['date'] = pd.to_datetime(volumes.date)

In [187]:
#get closing price day of headline
df2 = pd.merge(df, prices, on = ['ticker', 'date'], how = 'left', copy = False)
df2 = df2.rename(columns = {'close':'cur_day_close'})

#if cur_day_close is null, date might be on a holiday, offset those dates by 1 business day and try again
df2.loc[df2.cur_day_close.isnull(), 'date'] = df2.date - BDay(1)
df2 = pd.merge(df2, prices, on = ['ticker', 'date'], how = 'left', copy = False)

df2.loc[df2.cur_day_close.isnull(), 'cur_day_close'] = df2.close
del df2['close']

#remove rows where we do not have price data for on that date
df2 = df2[df2.cur_day_close.notnull()]

In [188]:
#remove rows with < 20 days of trading days
price_max_dates = prices.pivot_table(index = 'ticker', values = 'date', aggfunc = 'max').reset_index()
price_max_dates.columns = ['ticker', 'max_date']
df2 = pd.merge(df2, price_max_dates, on = 'ticker', how = 'left', copy = False)
df2 = df2[df2.date < df2.max_date - BDay(20)]

del df2['max_date']

In [148]:
#get previous day's close
df2 = df2.sort_values('date', ascending = True)
prices = prices.sort_values('date', ascending = True)

df2 = pd.merge_asof(df2, prices, by = 'ticker', left_on = 'date', right_on = 'date', direction = 'backward',
                    allow_exact_matches = False)

df2 = df2.rename(columns = {'close':'prev_day_close'})

#remove rows where we do not have price data on that previous date
df2 = df2[df2.prev_day_close.notnull()]

In [149]:
#calculate 10, 20, 50, 200 price price_dma
prices['price_5_dma'] = prices.groupby('ticker').close.rolling(5).mean().reset_index(0, drop = True)
prices['price_10_dma'] = prices.groupby('ticker').close.rolling(10).mean().reset_index(0, drop = True)
prices['price_20_dma'] = prices.groupby('ticker').close.rolling(20).mean().reset_index(0, drop = True)
prices['price_50_dma'] = prices.groupby('ticker').close.rolling(50).mean().reset_index(0, drop = True)
prices['price_100_dma'] = prices.groupby('ticker').close.rolling(100).mean().reset_index(0, drop = True)
prices['price_200_dma'] = prices.groupby('ticker').close.rolling(200).mean().reset_index(0, drop = True)

In [150]:
#join price price_dma to headlines
df2 = pd.merge(df2, prices[['ticker', 'date', 'price_5_dma','price_10_dma', 'price_20_dma', 'price_50_dma', 'price_100_dma','price_200_dma']],
               on = ['ticker', 'date'], how = 'left', copy = False)

In [151]:
#get price volumes day of headline
df2 = pd.merge(df2, volumes, on = ['ticker', 'date'], how = 'left', copy = False)
df2 = df2.rename(columns = {'volume':'cur_day_volume'})

In [152]:
#get previous day's volume
df2 = df2.sort_values('date', ascending = True)
volumes = volumes.sort_values('date', ascending = True)

df2 = pd.merge_asof(df2, volumes, by = 'ticker', left_on = 'date', right_on = 'date', direction = 'backward',
                    allow_exact_matches = False)

df2 = df2.rename(columns = {'volume':'prev_day_volume'})

In [153]:
#calculate 10, 20, 50, 200 volume volume_dma
volumes['volume_5_dma'] = volumes.groupby('ticker').volume.rolling(5).mean().reset_index(0, drop = True)
volumes['volume_10_dma'] = volumes.groupby('ticker').volume.rolling(10).mean().reset_index(0, drop = True)
volumes['volume_20_dma'] = volumes.groupby('ticker').volume.rolling(20).mean().reset_index(0, drop = True)
volumes['volume_50_dma'] = volumes.groupby('ticker').volume.rolling(50).mean().reset_index(0, drop = True)
volumes['volume_100_dma'] = volumes.groupby('ticker').volume.rolling(100).mean().reset_index(0, drop = True)
volumes['volume_200_dma'] = volumes.groupby('ticker').volume.rolling(200).mean().reset_index(0, drop = True)

In [154]:
#join volume_dma to headlines
df2 = pd.merge(df2, volumes[['ticker', 'date', 'volume_5_dma','volume_10_dma', 'volume_20_dma', 'volume_50_dma', 'volume_100_dma','volume_200_dma']],
               on = ['ticker', 'date'], how = 'left', copy = False)

In [155]:
#get % price movement from previous day
df2['price_move'] = (df2.cur_day_close - df2.prev_day_close) / df2.prev_day_close

In [156]:
df2_copy = df2.copy(deep = True)

In [179]:
df2 = df2_copy.copy(deep = True)

In [180]:
#get price after 19 days from current headline date (20 trading days from previous close date)
df2['end_date'] = df2.date + BDay(19)
df2 = pd.merge_asof(df2, prices[['ticker', 'date', 'close']], by = 'ticker', left_on = 'end_date', right_on = 'date', direction = 'forward',
                    allow_exact_matches = True)

del df2['end_date']
del df2['date_y']

df2 = df2.rename(columns = {'date_x':'date', 'close':'period_end_close'})

In [181]:
#save dataframe of cleaned stock headlines
df2.to_csv('stock_headlines_cleaned.csv', index = False)